In [ ]:
import requests
import urllib.request as req
from bs4 import BeautifulSoup

In [ ]:
url = 'https://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108'

res = req.urlopen(url)
soup = BeautifulSoup(res,"html.parser")
info = {}

for i in soup.find_all("location"):
    print(i.find("city"))

In [ ]:
# 서울로 테스트
seoul = soup.find("location")
# tmef: 날짜, wf: 날씨, tmn: 최저기온, tmx: 최고기온
for i in seoul.find_all("data"):
    print(i.find("tmef").text)
    print(i.find("wf").text)
    print(i.find("tmn").text)
    print(i.find("tmx").text)
    

In [ ]:
# 도시별 날씨 데이터 구축
weather = {}
for i in soup.find_all("location"):
    weather[i.find('city').text] = []
    for j in i.find_all('data'):
        temp = []
        temp.append(j.find('tmef').text)
        temp.append(j.find('wf').text)
        temp.append(j.find('tmn').text)
        temp.append(j.find('tmx').text)
        weather[i.find('city').text].append(temp)

In [ ]:
# 서울의 일주일치 날씨 data
print(weather['서울'])

In [ ]:
# 제주의 일주일치 날씨 data
print(weather['제주'])

In [ ]:
# csv file로 저장하기
# 현재 dictionary로 되어있으나 csv로 저장하기 위해 list로 바꿔야 한다.
for i in weather:
    for j in weather[i]:
        print(i, j[0],j[1],j[2],j[3])
    print("-"*30)


In [ ]:
# List로 생성하기
tempList = []
for i in weather:
    for j in weather[i]:
        tempList.append((i, j[0],j[1],j[2],j[3]))
tempList


In [ ]:
import pandas as pd

In [ ]:
weatherDataFrame = pd.DataFrame(tempList,columns=['지역','날짜','날씨','최저온도','최고온도'])
weatherDataFrame

In [ ]:
weatherDataFrame.to_csv("../Data/weatherList.csv",index=False)

In [ ]:
forecast = pd.read_csv("../Data/weatherList.csv")
forecast

In [ ]:
# 파이썬 분석할 때 쓰는 코드
forecast.describe()
# 문자열은 나오지 않는다.
# count, mean, std, min, max, 4분위

---
# Sqlite로 저장

In [ ]:
import sqlite3

In [ ]:
# 저장경로 지정
filePath = "../Data/forecast.sqlite"
conn = sqlite3.connect(filePath)

# 테이블을 생성하고 데이터 넣기
curs = conn.cursor()
# executescript는 여러개 실행할 때
# 하나라면 execute
curs.executescript(
    """
    /* items 테이블이 이미 있다면 제거하기 */
    DROP TABLE IF EXISTS forecast;

    /* 테이블 생성하기 */
    CREATE TABLE forecast(
        seq INTEGER PRIMARY KEY AUTOINCREMENT,
        city TEXT,
        datetime TEXT,
        weather TEXT,
        min DOUBLE,
        max DOUBLE
    );
    """
)

# 위의 작업을 데이터베이스에 반영하기
conn.commit()


In [ ]:
# 데이터 넣기
forecast = pd.read_csv("../Data/weatherList.csv")
forecast_dict = forecast.to_dict()
city_list = list(forecast_dict['지역'].values())
date_list = list(forecast_dict['날짜'].values())
weather_list = list(forecast_dict['날씨'].values())
min_list = list(forecast_dict['최저온도'].values())
max_list = list(forecast_dict['최고온도'].values())

# print(range(len(forecast)))
for i in range(len(forecast)):
    curs.execute("INSERT INTO forecast(city, datetime, weather, min, max) VALUES (?,?,?,?,?)",(city_list[i],date_list[i],weather_list[i],min_list[i],max_list[i]))    
    conn.commit()

In [ ]:
# 데이터 들어갔는지 확인하기
curs.execute("SELECT * FROM forecast")
forecast_list = curs.fetchall()
# print(forecast_list)
for i in forecast_list:
    print(i[0],'\t',i[1],'\t',i[2],i[3],'    ',i[4],i[5])

---
# MySQL로 저장

In [142]:
import pymysql

In [143]:
conn = pymysql.connect(
    host = "127.0.0.1",
    user = "root",
    password = "qwer1234",
    db = "python",
    charset = "utf8"
)

In [145]:
# 쿼리를 실행할 커서 생성
curs = conn.cursor()

In [146]:
select = "select * from forecast"
curs.execute(select)
rows = curs.fetchall()
row_list = list(rows)
print(row_list)


[(1, '0    서울\nName: 지역, dtype: object', '0    2023-01-23 00:00\nName: 날짜, dtype: object', '0    구름많음\nName: 날씨, dtype: object', -3.0, -1.0), (2, '1    서울\nName: 지역, dtype: object', '1    2023-01-23 12:00\nName: 날짜, dtype: object', '1    맑음\nName: 날씨, dtype: object', -3.0, -1.0), (3, '2    서울\nName: 지역, dtype: object', '2    2023-01-24 00:00\nName: 날짜, dtype: object', '2    맑음\nName: 날씨, dtype: object', -17.0, -10.0), (4, '3    서울\nName: 지역, dtype: object', '3    2023-01-24 12:00\nName: 날짜, dtype: object', '3    맑음\nName: 날씨, dtype: object', -17.0, -10.0), (5, '4    서울\nName: 지역, dtype: object', '4    2023-01-25 00:00\nName: 날짜, dtype: object', '4    맑음\nName: 날씨, dtype: object', -16.0, -6.0), (6, '5    서울\nName: 지역, dtype: object', '5    2023-01-25 12:00\nName: 날짜, dtype: object', '5    맑음\nName: 날씨, dtype: object', -16.0, -6.0), (7, '6    서울\nName: 지역, dtype: object', '6    2023-01-26 00:00\nName: 날짜, dtype: object', '6    흐림\nName: 날씨, dtype: object', -11.0, -2.0), (8, '7    서울\nNam

In [96]:
insert = "insert into forecast(city,date,weather,min,max) values(%s,%s,%s,%s,%s)"

In [157]:
forecast = pd.read_csv("../Data/weatherList.csv")
forecast.describe()
# count = 533

# 지역,날짜,날씨,최저온도,최고온도

for i in range(len(forecast)):
    city = forecast.loc[i][0]
    date = forecast.loc[i][1]
    weather = forecast.loc[i][2]
    min = float(forecast.loc[i][3])
    max = float(forecast.loc[i][4])
    curs.execute(insert,(city,date,weather,float(min),float(max)))

    conn.commit()